In [ ]:
# pip install langchain qdrant_client openai streamlit

In [ ]:
### DO NOT RUN ALL ###

In [ ]:
### these are functions to run individually as needed to modify the Qdrant collections!! the py. files are to run the script ###

In [ ]:
# Imports
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from qdrant_client import QdrantClient, models
import os
import qdrant_client
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from PyPDF2 import PdfReader

In [ ]:
# create qdrant client
client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"], 
        api_key=st.secrets["QDRANT_API_KEY"]
    )

In [ ]:
# delete collection

client.delete_collection(collection_name=st.secrets["QDRANT_COLLECTION_NAME"])

In [ ]:
# create collection

# define parameters for vectors
vectors_config = models.VectorParams(
    size=1536, # vector size for OpenAI
    distance=models.Distance.COSINE
)

# create collection
client.recreate_collection(
    collection_name=st.secrets["QDRANT_COLLECTION_NAME"],
    vectors_config=vectors_config,
)

In [ ]:
# create vector store

embeddings = OpenAIEmbeddings()

# get collection name
vector_store = Qdrant(
    client=client, 
    collection_name=st.secrets["QDRANT_COLLECTION_NAME"], 
    embeddings=embeddings,
)

In [ ]:
# get chunks function
def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
# get pdf text function
def get_pdf_text(pdf_filepath):
    text = ""
    with open(pdf_filepath, "rb") as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()  # Or use page.extractText() if using an older version of PyPDF2
    return text

In [29]:
# get txt text function
def get_txt_text(txt_filepath):
    with open(txt_filepath, 'r', encoding='latin1') as file:
        text = file.read()
    return text

In [33]:
# upload slides to vector store
# get filepath
slides_filepath = 'Slides'
for slide in os.listdir(slides_filepath):
    slide_path = os.path.join(slides_filepath, slide)
    # extract text
    texts = get_pdf_text(slide_path)
    chunks = get_chunks(texts)
    metadatas = [{"name": slide} for _ in texts]
    vector_store.add_texts(chunks, metadatas=metadatas)
    


FloatObject (b'0.00-29555193') invalid; use 0.0 instead
FloatObject (b'0.00-22356359') invalid; use 0.0 instead
FloatObject (b'0.00-25406823') invalid; use 0.0 instead
FloatObject (b'0.000-29396326') invalid; use 0.0 instead
FloatObject (b'0.0000-52493438') invalid; use 0.0 instead
FloatObject (b'0.0000-52493438') invalid; use 0.0 instead
FloatObject (b'0.00-50') invalid; use 0.0 instead
FloatObject (b'0.00-50') invalid; use 0.0 instead
FloatObject (b'0.0000-13123359') invalid; use 0.0 instead
FloatObject (b'0.00-60') invalid; use 0.0 instead
FloatObject (b'0.00-60') invalid; use 0.0 instead
FloatObject (b'0.00-60') invalid; use 0.0 instead
FloatObject (b'0.00-60') invalid; use 0.0 instead


In [34]:
def upload_txt_folder(folder_filepath):
    for file_name in os.listdir(folder_filepath):
        filepath = os.path.join(folder_filepath, file_name)
        texts = get_txt_text(filepath)
        chunks = get_chunks(texts)
        metadatas = [{"name": file_name} for _ in texts]
        vector_store.add_texts(chunks, metadatas=metadatas)
    


In [35]:
upload_txt_folder('Output')